# ((2,2))
Let consider the simplest case ((2,2)). In this case, Alice sends two receivers Bob and Charlie a quantum state. When and only when Bob and Charlie work together, they can recover the secret message from Alice. Neither Bob nor Charlie can extract Alice’s secret on their own.

In [7]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
import random

In [2]:
def QuantumSecretSharing(Alice, Bob, Charlie):
    # Create registers
    q = QuantumRegister(3, 'q')
    c = ClassicalRegister(3, 'c')

    # Create circuit
    circuit = QuantumCircuit(q, c)

    # Prepare the state
    circuit.h(q[0])
    circuit.cx(q[0], q[1])
    circuit.cx(q[0], q[2])

    # Alice randomly chooses X- or Y-basis and measures
    if Alice == 'X':
        circuit.h(q[0])
        circuit.measure(q[0], c[0])
    if Alice == 'Y':
        circuit.sdg(q[0])
        circuit.h(q[0])
        circuit.measure(q[0], c[0])
    
    # Bob randomly chooses X- or Y-basis and measures
    if Bob == 'X':
        circuit.h(q[1])
        circuit.measure(q[1], c[1])
    if Bob == 'Y':
        circuit.sdg(q[1])
        circuit.h(q[1])
        circuit.measure(q[1], c[1])
    
    # Charlie randomly chooses X- or Y-basis and measures
    if Charlie == 'X':
        circuit.h(q[2])
        circuit.measure(q[2], c[2])
    if Charlie == 'Y':
        circuit.sdg(q[2])
        circuit.h(q[2])
        circuit.measure(q[2], c[2])
    
    # Execute the circuit
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=1)
    result = job.result()
    counts = result.get_counts(circuit)
    result = list(counts.keys())[0] # a string like '101'
    Alice_result = int(result[0])
    Bob_result = int(result[1])
    Charlie_result = int(result[2])


    print(f'Alice chooses {Alice}, Bob chooses {Bob}, Charlie chooses {Charlie}')
    if (Alice == Bob and Charlie == 'Y') or (Alice != Bob and Charlie == 'X'):
        print('Cannot determine the secret!')
        return
    if Alice == Bob and Alice == 'X':
        print(f'Charlie\'s result is {Charlie_result}')
        if Charlie_result == 0:
            print(f'Charlie knows Bob\'s result is equal to Alice\'s result')
            print(f'Bob\'s result is {Bob_result}')
            guess_result = Bob_result
            print(f'So they can recover Alice\'s result: {guess_result}')
        else:
            print(f'Charlie knows Bob\'s result is different from Alice\'s result')
            print(f'Bob\'s result is {Bob_result}')
            guess_result = 1 if Bob_result == 0 else 0
            print(f'So they can recover Alice\'s result: {guess_result}')
    else:
        print(f'Charlie\'s result is {Charlie_result}')
        if Charlie_result == 0:
            print(f'Charlie knows Bob\'s result is different from Alice\'s result')
            print(f'Bob\'s result is {Bob_result}')
            guess_result = 1 if Bob_result == 0 else 0
            print(f'So they can recover Alice\'s result: {guess_result}')
        else:
            print(f'Charlie knows Bob\'s result is equal to Alice\'s result')
            print(f'Bob\'s result is {Bob_result}')
            guess_result = Bob_result
            print(f'So they can recover Alice\'s result: {guess_result}')
    print(f'Alice\'s really result is {Alice_result}')
    if guess_result == Alice_result:
        print('They succeed!')
    else:
        print('They fail!')

In [6]:
for i in range(10):
    print(f'--------------------Round {i+1}--------------------')
    Alice_measure_basis = random.choice(['X', 'Y'])
    Bob_measure_basis = random.choice(['X', 'Y'])
    Charlie_measure_basis = random.choice(['X', 'Y'])
    QuantumSecretSharing(Alice_measure_basis, Bob_measure_basis, Charlie_measure_basis)
    print('\n')

--------------------Round 1--------------------
Alice chooses Y, Bob chooses X, Charlie chooses X
Cannot determine the secret!


--------------------Round 2--------------------
Alice chooses Y, Bob chooses X, Charlie chooses X
Cannot determine the secret!


--------------------Round 3--------------------
Alice chooses X, Bob chooses Y, Charlie chooses Y
Charlie's result is 0
Charlie knows Bob's result is different from Alice's result
Bob's result is 0
So they can recover Alice's result: 1
Alice's really result is 1
They succeed!


--------------------Round 4--------------------
Alice chooses X, Bob chooses Y, Charlie chooses X
Cannot determine the secret!


--------------------Round 5--------------------
Alice chooses X, Bob chooses Y, Charlie chooses X
Cannot determine the secret!


--------------------Round 6--------------------
Alice chooses Y, Bob chooses X, Charlie chooses Y
Charlie's result is 0
Charlie knows Bob's result is different from Alice's result
Bob's result is 1
So they